# EDA for FAA data
Original download is an Excel file.
Typical running times:
- from scratch: ~ 8 mins if not saving to Excel
- from uploaded gzip file: < 1 min if not saving to Excel

In [ ]:
must_save_final_to_excel = False  # True adds about 8 minutes to runtime

In [ ]:
import datetime
from datetime import datetime, date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

start_time = datetime.now()
print (f'Start: {start_time}')

In [ ]:
# If Output/df_main1.parquet.gzip exists, load that otherwise load from Excel
raw_data_file_path = 'Data/Wildlife strike data.xlsx'
parquet_file_path = 'Output/df_main1.parquet.gzip'
is_data_loaded_from_Excel = False
try:
    # Takes less than 1 second to read in approx 30MB of 327K rows
    df_main = pd.read_parquet(parquet_file_path)
    print(f'Loaded cleaned data from {parquet_file_path}')
except FileNotFoundError:
    # Takes 6 to 7 minutes to read in approx 140MB of 327K rows
    is_data_loaded_from_Excel = True
    df_main = pd.read_excel(raw_data_file_path, sheet_name='Sheet1', dtype=str)
    print(f'Loaded raw data from {raw_data_file_path}')

In [ ]:
if is_data_loaded_from_Excel:
    # Numeric columns
    df_main.describe()

In [ ]:
# Non-numeric columns
if is_data_loaded_from_Excel:
    df_main.describe(include=['O'])

In [ ]:
if is_data_loaded_from_Excel:
    df_main.info()

In [ ]:
if is_data_loaded_from_Excel:
    # Drop columns of no interest
    drop_cols = ['RUNWAY', 'LOCATION', 'OPID', 'REG', 'FLT', 'AMA', 'AMO', 'EMA', 'EMO', 'ENG_1_POS', 'ENG_2_POS', 'ENG_3_POS', 'ENG_4_POS', 'AOS', 
                'COST_REPAIRS', 'COST_OTHER', 'COST_REPAIRS_INFL_ADJ', 'COST_OTHER_INFL_ADJ', 'INGESTED_OTHER', 'STR_OTHER', 'DAM_OTHER', 
                'OTHER_SPECIFY', 'EFFECT_OTHER', 'BIRD_BAND_NUMBER', 'SPECIES_ID',
                'REMARKS', 'ENROUTE_STATE', 'NR_INJURIES', 'NR_FATALITIES', 'COMMENTS', 'REPORTED_NAME', 'REPORTED_TITLE',
                'SOURCE', 'PERSON', 'LUPDATE', 'TRANSFER'
    ]
    for col in drop_cols:
        if col in df_main.columns:
            df_main.drop(columns=[col], inplace=True)

    df_main.describe()

In [ ]:
if is_data_loaded_from_Excel:
    df_main.describe(include=['O'])

In [ ]:
if is_data_loaded_from_Excel:
    df_main.info()

In [ ]:
if is_data_loaded_from_Excel:
    # Drop rows with non-numeric LATITUDE or LONGITUDE
    df_main = df_main[pd.to_numeric(df_main['AIRPORT_LATITUDE'], errors='coerce').notnull()]
    df_main = df_main[pd.to_numeric(df_main['AIRPORT_LONGITUDE'], errors='coerce').notnull()]
    # Convert NUM_SEEN and NUM_STRUCK to strings
    df_main['NUM_SEEN'] = df_main['NUM_SEEN'].astype(str)
    df_main['NUM_STRUCK'] = df_main['NUM_STRUCK'].astype(str)

In [ ]:
if is_data_loaded_from_Excel:
    # Find columns with inconsistent data types. Ignore columns with all numeric or all non-numeric values. 
    # Ignore columns with mixed types but no numeric values. Ignore columns with missing values but otherwise consistent types.
    # Get counts of different types vs columns.
    type_counts = {}
    for col in df_main.columns:
        types = df_main[col].apply(lambda x: type(x)).value_counts()
        if len(types) > 1 and not (types.index.isin([int, float]).all() or types.index.isin([str]).all() or (types.index.isin([str, type(None)]).all()) or (types.index.isin([int, float, type(None)]).all())):
            type_counts[col] = types
    type_counts_df = pd.DataFrame(type_counts).fillna(0).astype(int)

    # Transpose for display purposes.
    type_counts_df = type_counts_df.T

    print(type_counts_df)

In [ ]:
if is_data_loaded_from_Excel:
    # Identify columns with missing values. Give counts and percentages to 1 decimal place.
    missing_counts = df_main.isnull().sum()
    missing_percent = ( missing_counts / len(df_main) * 100 ).round(1)
    missing_df = pd.DataFrame({'Missing Count': missing_counts, 'Missing Percent': missing_percent})
    missing_df = missing_df[missing_df['Missing Count'] > 0]
    print(missing_df)

In [ ]:
if is_data_loaded_from_Excel:
    # Remove columns with more than 50% missing values
    cols_to_drop = missing_df[missing_df['Missing Percent'] > 50].index
    df_main.drop(columns=cols_to_drop, inplace=True)
    print(f'Dropped columns with > 50% missing values: {list(cols_to_drop)}')

In [ ]:
# Final checks of data (place all other data cleaning above this line)
# Check 1/3
df_main.info()

In [ ]:
# Check 2/3
df_main.describe()

In [ ]:
# Check 3/3
df_main.describe(include=['O'])

In [ ]:
# Set integer columns to integer data types
int_columns = ['INDEX_NR', 'INCIDENT_MONTH', 'INCIDENT_YEAR', 'AC_MASS', 'NUM_ENGS', 'NR_INJURIES', 'NR_FATALITIES']
for col in int_columns:
    if col in df_main.columns:
        df_main[col] = df_main[col].astype('Int64')


In [ ]:
output_path_1 = 'Output/df_main1.parquet.gzip'

if is_data_loaded_from_Excel:
    df_main.to_parquet(output_path_1, compression='gzip')
# print current time and elapsed time
end_time = datetime.now()
# Print output path
print(f'Cleaned data saved to {output_path_1}\n')

print(
    f'Load, clean and save data end: {datetime.strftime(end_time, "%H:%M:%S")},'
    + f' Elapsed time: {str(end_time - start_time).split(".")[0]}'
)


## Add derived columns

In [ ]:
# Add an index column based on FAAREGION, call it FAAREGION_INDEX
if 'FAAREGION' in df_main.columns:
    df_main['FAAREGION'] = df_main['FAAREGION'].astype(str).str.strip()
    faa_region_mapping = {region: idx for idx, region in enumerate(sorted(df_main['FAAREGION'].unique()), start=0)}
    df_main['FAAREGION_INDEX'] = df_main['FAAREGION'].map(faa_region_mapping).astype('Int64')

In [ ]:
# Split day into half-hour periods. Time is given as HH:MM. Ignore missing values
def split_time_to_half_hour_periods(time_str):
    if pd.isna(time_str):
        return np.nan
    try:
        time_obj = datetime.strptime(time_str, '%H:%M')
        hour = time_obj.hour
        minute = time_obj.minute
        if minute < 30:
            return f'{hour:02d}:00-{hour:02d}:29'
        else:
            # next_hour = (hour + 1) % 24
            return f'{hour:02d}:30-{hour:02d}:59'
    except ValueError:
        return np.nan

df_main['TIME_PERIOD'] = df_main['TIME'].apply(split_time_to_half_hour_periods)
# Check TIME_PERIOD
df_main['TIME_PERIOD'].value_counts(dropna=False).sort_index()

In [ ]:
# Assign a time period index to each half-hour period
def time_period_index(time_period_str):
    if pd.isna(time_period_str):
        return np.nan
    try:
        start_time = time_period_str.split('-')[0]
        hour, minute = map(int, start_time.split(':'))
        return hour * 2 + (1 if minute >= 30 else 0)
    except ValueError:
        return np.nan

df_main['TIME_PERIOD_INDEX'] = df_main['TIME_PERIOD'].apply(time_period_index)
# Check TIME_PERIOD_INDEX
df_main['TIME_PERIOD_INDEX'].value_counts(dropna=False).sort_index()
df_main['TIME_PERIOD_INDEX'] = df_main['TIME_PERIOD_INDEX'].astype('Int64')

In [ ]:
df_main['INCIDENT_HOUR'] = df_main['TIME'].apply(lambda x: x.split(':')[0] if ':' in str(x) else np.nan).astype('Int64')
df_main['INCIDENT_MINUTE'] = df_main['TIME'].apply(lambda x: x.split(':')[1] if ':' in str(x) else np.nan).astype('Int64')
df_main['INCIDENT_HALF_HOUR'] = df_main.apply(lambda row:
    f"{int(row['INCIDENT_HOUR']):02d}:{'00' if row['INCIDENT_MINUTE'] < 30 else '30'}" 
        if pd.notnull(row['INCIDENT_HOUR']) and pd.notnull(row['INCIDENT_MINUTE']) 
        else np.nan,
    axis=1
)

In [ ]:
# Derive abbreviated month name from INCIDENT_MONTH
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df_main['INCIDENT_MONTH_NAME'] = df_main['INCIDENT_MONTH'].apply(lambda x: month_names.get(x) if pd.notnull(x) else np.nan)
# Check INCIDENT_MONTH_NAME, sort by month number
df_main[['INCIDENT_MONTH', 'INCIDENT_MONTH_NAME']].drop_duplicates().sort_values(by='INCIDENT_MONTH')


In [ ]:
# Assign day of year to INCIDENT_DATE
df_main['INCIDENT_DAY_OF_YEAR'] = pd.to_datetime(df_main['INCIDENT_DATE'], errors='coerce').dt.dayofyear.astype('Int64')
print(df_main['INCIDENT_DAY_OF_YEAR'].max())
# # Decrement INCIDENT_DAY_OF_YEAR when INCIDENT_DATE is after 29th February and year is a leap year
# df_main['INCIDENT_DAY_OF_YEAR'] = df_main.apply(lambda row: row['INCIDENT_DAY_OF_YEAR'] - 1
#                                                 if pd.notnull(row['INCIDENT_DAY_OF_YEAR']) and
#                                                    pd.notnull(row['INCIDENT_YEAR']) and
#                                                    row['INCIDENT_YEAR'] % 4 == 0 and
#                                                    row['INCIDENT_DAY_OF_YEAR'] > 59
#                                                 else row['INCIDENT_DAY_OF_YEAR'], axis=1)
# print(df_main['INCIDENT_DAY_OF_YEAR'].max())

In [ ]:
# Change FAAREGION column name to FAA Region and all other column names to Title Case with spaces instead of underscores
df_main.columns = [col.replace('_', ' ').title() for col in df_main.columns]
df_main.rename(columns={'Faaregion': 'FAA Region', 'Faaregion Index': 'FAA Region Index'}, inplace=True)

In [ ]:
# Save cleaned and prepared data as Parquet gzip
output_path_2_parquet = 'Output/df_main2.parquet.gzip'
df_main.to_parquet(output_path_2_parquet, compression='gzip')
# Print output path
print(f'{datetime.strftime(datetime.now(), "%H:%M:%S")} Additional data saved to {output_path_2_parquet}\n')

In [ ]:
# Save cleaned and prepared data as csv
output_path_2_csv = 'Output/Wildlife Cleaned.csv'
df_main.to_csv(output_path_2_csv, index=False)
# Print output path
print(f'{datetime.strftime(datetime.now(), "%H:%M:%S")} Additional data saved to {output_path_2_csv}\n')

In [ ]:
# Save cleaned and prepared data as Excel
if must_save_final_to_excel:
    # This step takes circa 8 mins for 327K rows
    output_path_2_excel = 'Output/Wildlife Cleaned.xlsx'
    df_main.to_excel(output_path_2_excel, index=False)
    # Print output path
    print(f'{datetime.strftime(datetime.now(), "%H:%M:%S")} Additional data saved to {output_path_2_excel}\n')

In [ ]:
# print current time and elapsed time
end_time = datetime.now()
print(
    f'Save cleaned and prepared data end: {datetime.strftime(end_time, "%H:%M:%S")},'
    + f' Elapsed time: {str(end_time - start_time).split(".")[0]}'
)

In [ ]:
# Measure columns
measure_cols = ['INCIDENT_MONTH', 'AC_MASS', 'NUM_ENGS', 'HEIGHT', 'TIME_PERIOD_INDEX']
# Make column names Title Case with spaces instead of underscores
measure_cols = [col.replace('_', ' ').title() for col in measure_cols]


In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(df_main[measure_cols].corr(), vmax=0.6, square=True, annot=True)

AC_MASS and NUM_ENGS:
- no surprise here - the bigger the aircraft, the more engines it is likely to have. Slightly surprised the correlation is not stronger

Everything else:
- uncorrelated

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(measure_cols):
    if pd.api.types.is_numeric_dtype(df_main[col]):
        q1 = df_main[col].quantile(0.25)
        q3 = df_main[col].quantile(0.75)
        
        sns.histplot(df_main[col], kde=True, ax=axes[i], color='skyblue', edgecolor='black')
        axes[i].axvline(q1, color='red', linestyle='--', label=f'Q1 (25%): {q1:.2f}')
        axes[i].axvline(q3, color='green', linestyle='--', label=f'Q3 (75%): {q3:.2f}')
        axes[i].legend()
        
    else:
        sns.countplot(
            data=df_main, 
            x=col, 
            hue=col,       
            ax=axes[i], 
            palette='cool', 
            edgecolor='black', 
            legend=False   
        )   
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].set_title(f'Distribution of {col}', fontsize=14)
    axes[i].set_xlabel(col, fontsize=12)
    axes[i].set_ylabel('Frequency', fontsize=12)


plt.tight_layout()
plt.show()

In [ ]:
end_time = datetime.now()

elapsed_time = end_time - start_time
print (f"Elapsed from start: {str(elapsed_time).split('.')[0]}; Time: {datetime.now().strftime(format='%Y-%m-%d %H:%M:%S')}")